In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from ipywidgets import FloatSlider
import pandas as pd
import sklearn
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import OLSInfluence
import numpy as np
import seaborn as sns
import shap
import matplotlib.pyplot as plt

In [ ]:
df_ = pd.read_csv("https://raw.githubusercontent.com/thistleknot/Python-Stock/master/data/raw/states.csv") 

In [ ]:
df = sm.add_constant(df_)[[*df_.columns,'const']]

In [ ]:
df

In [ ]:
#df[['Poverty', 'White', 'Traf Deaths', 'University', 'Unemployed', 'Income', 'Population']]

In [139]:
def return_model(x,y):
    #print(x)
    names = [y,*x]
    print(names)
    #print()
    shap.initjs()
    
    data = df[names].sort_values(kind="quicksort", by=names[0], ascending=True,key=abs)
    X = data.iloc[:,1:]
    Y = data.iloc[:,0]
    
    model = sklearn.linear_model.LinearRegression()
    
    model.fit(X, Y)
    model_ = sm.OLS(Y,X)
    results = model_.fit()
    #shap
    background = np.array(X)
    e = shap.LinearExplainer(model, X)
    
    shap_values = e.shap_values(np.array(X))
    shap.summary_plot(shap_values, -np.array(X))
    explainer = shap.Explainer(model, X)
    shap.plots.heatmap(explainer(X))
    
    Y_sort = Y.sort_values(kind="quicksort", ascending=True,key=abs)
    predict = results.predict(X.loc[Y_sort.index])
    plt.plot(np.array(Y_sort))
    plt.plot(np.array(predict))
    
    #return([data,results.summary()])
    return(results.summary())

def derive_xnames(y):
    x_ = widgets.SelectMultiple(
        options=df.columns[~df.columns.isin([y])][1:],
        value=[df.columns[~df.columns.isin([y])][1]],
        disabled=False
    )
    subset = interact(return_model,x=x_,y=y)

    return(subset)

y=y=widgets.Select(options=df.columns[1:],disabled=False)

a=interact(derive_xnames,y=y)



interactive(children=(Select(description='y', options=('Poverty', 'Infant Mort', 'White', 'Crime', 'Doctors', …